The purpose of this notebook is to merge the ACS data with the 2019 TIGER/Line shapefile of Census Tracts in New York. This enables further merging and cross-analysis with the georeferenced Yelp data. 

In [1]:
# Setting up modules
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# Data path 
path_census_tracts = './data/ACS/'
path_yelp = './data/Yelp/'

In [2]:
# Import data
NY_state_census_tracts = gpd.read_file(path_census_tracts + "CensusTracts/tl_2019_36_tract.shp")
MN_Yelp = gpd.read_file(path_yelp + "MN/MN_Yelp.shp")
BK_Yelp = gpd.read_file(path_yelp + "BK/BK_Yelp.shp")
BK_hh_income_dis_pct = pd.read_csv(path_census_tracts + 'BK_hh_income_dis_pct.csv')
BK_hh_income_dis_pct = pd.DataFrame(BK_hh_income_dis_pct)
NYC_neighborhood_tabn_areas = gpd.read_file(path_census_tracts + "CensusTracts/nynta_reproj.shp")

In [3]:
NYC_neighborhood_tabn_areas.head()

,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng,Shape_Area,geometry
0,3,Brooklyn,047,BK88,Borough Park,39247.228074,5.400502e+07,POLYGON ((-73.97604935657382 40.63127590564677...
1,4,Queens,081,QN51,Murray Hill,33266.904811,5.248828e+07,POLYGON ((-73.80379022888246 40.77561011179247...
2,4,Queens,081,QN27,East Elmhurst,19816.711538,1.972685e+07,POLYGON ((-73.86109724401859 40.76366447708767...
3,4,Queens,081,QN07,Hollis,20976.335837,2.288777e+07,POLYGON ((-73.75725671509139 40.71813860166255...
4,1,Manhattan,061,MN06,Manhattanville,17040.686548,1.064708e+07,"POLYGON ((-73.94607828608069 40.8212632160616,..."


In [75]:
BK_hh_income_dis_pct.head()

,ID,Geography,Households,0-25k,25k-50k,50k-75k,75k-100k,100k-125k,125k-150k,> 150k
0,36047000100,"Census Tract 1, Kings County, New York",2184,0.212,0.099,0.087,0.184,0.066,0.070,0.281
1,36047000200,"Census Tract 2, Kings County, New York",377,0.263,0.305,0.207,0.069,0.125,0.000,0.032
2,36047000301,"Census Tract 3.01, Kings County, New York",1865,0.043,0.205,0.009,0.107,0.124,0.076,0.435
3,36047000501,"Census Tract 5.01, Kings County, New York",1772,0.188,0.042,0.094,0.056,0.095,0.117,0.407
4,36047000502,"Census Tract 5.02, Kings County, New York",1560,0.132,0.115,0.074,0.107,0.099,0.072,0.399


In [4]:
# Filter for NYC (Manhattan and Brooklyn)

BK = NY_state_census_tracts[NY_state_census_tracts["COUNTYFP"] == '047']
MN = NY_state_census_tracts[NY_state_census_tracts["COUNTYFP"] == '061']
BK_neighborhood_tabn_areas = NYC_neighborhood_tabn_areas[NYC_neighborhood_tabn_areas["BoroName"] == "Brooklyn"]
MN_neighborhood_tabn_areas = NYC_neighborhood_tabn_areas[NYC_neighborhood_tabn_areas["BoroName"] == "Manhattan"]

# Cutting out all of the crap
MN = MN[["GEOID", "geometry"]]
BK = BK[["GEOID", "geometry"]]
BK_neighborhood_tabn_areas = BK_neighborhood_tabn_areas[["NTAName", "geometry"]]
MN_neighborhood_tabn_areas = MN_neighborhood_tabn_areas[["NTAName", "geometry"]]

# Converting type of column
# int wont work because it is too long
BK.GEOID = BK.GEOID.astype(float)
# Merging
BK = BK.merge(BK_hh_income_dis_pct, left_on = 'GEOID', right_on = 'ID')

In [5]:
# Reprojecting the BK_NTA file
#BK_neighborhood_tabn_areas.crs = {'init': 'epsg:4269'}
#MN_neighborhood_tabn_areas.crs = {'init': 'epsg:4269'}

# Are the CRSes identical?
print(BK.crs == BK_Yelp.crs)
print(BK.crs == BK_neighborhood_tabn_areas.crs)

True
True


In [7]:
BK_neighborhood_tabn_areas.geometry

0      POLYGON ((-73.97604935657382 40.63127590564677...
6      POLYGON ((-73.95859278495767 40.61040303093504...
12     POLYGON ((-73.93753749374042 40.60855739025752...
13     POLYGON ((-73.97084113663696 40.64637857107214...
14     POLYGON ((-73.94826499590911 40.63860718970649...
16     POLYGON ((-73.95337017508862 40.68064050844432...
21     POLYGON ((-73.96014773493863 40.62891518541702...
22     POLYGON ((-73.96514385192494 40.59110191611803...
23     POLYGON ((-73.97477657908587 40.61263847492669...
24     POLYGON ((-73.95023693757913 40.70547324665451...
25     POLYGON ((-73.94193078816194 40.70072523469546...
26     POLYGON ((-73.91804606958485 40.68721324777091...
27     POLYGON ((-73.90404639808889 40.67922059801916...
28     POLYGON ((-73.92164748131954 40.67887054703262...
29     POLYGON ((-73.96131877957791 40.67140667693574...
30     POLYGON ((-73.90855790456779 40.65209593779397...
32     (POLYGON ((-73.88828531355972 40.6467224137786...
33     POLYGON ((-73.9257438942

In [8]:
# Spatial join for Yelp to Census Tract
MN_Yelp_joined = gpd.sjoin(MN_Yelp, MN, how = 'inner', op = 'within')
MN_Yelp_joined = MN_Yelp_joined.drop(columns = ["index_right"])
BK_Yelp_joined = gpd.sjoin(BK_Yelp, BK, how = 'inner', op = 'within')
BK_Yelp_joined = BK_Yelp_joined.drop(columns = ["index_right"])

# Spatial join for Yelp and Census Tract to Neighborhood Tabulation Area 
BK_Yelp_NTA_joined = gpd.sjoin(BK_Yelp_joined, BK_neighborhood_tabn_areas, how = "inner", op = "within")
MN_Yelp_NTA_joined = gpd.sjoin(MN_neighborhood_tabn_areas, MN_Yelp_joined, how = "inner", op = "within")
BK_Yelp_NTA_joined.head(2)

C:\Users\wesch\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


,id,alias,name,is_closed,review_cou,rating,price,categories,latitude,longitude,...,Households,0-25k,25k-50k,50k-75k,75k-100k,100k-125k,125k-150k,> 150k,index_right,NTAName
0,6gzQLjzJk25ePm_JS7ZAug,esme-brooklyn-2,Esme,0,328,4.5,$$,newamerican|cocktailbars,40.733203,-73.954967,...,2250,0.285,0.114,0.13,0.116,0.084,0.084,0.186,170,Greenpoint
2,utM-5navObsVA5sCRHobzA,madre-brooklyn-2,Madre,0,38,5.0,MISSING,newamerican,40.733110,-73.957980,...,2250,0.285,0.114,0.13,0.116,0.084,0.084,0.186,170,Greenpoint


In [10]:
BK_Yelp_NTA_joined.columns

Index(['id', 'alias', 'name', 'is_closed', 'review_cou', 'rating', 'price',
       'categories', 'latitude', 'longitude', 'address', 'city', 'zipcode',
       'state', 'country', 'geometry', 'GEOID', 'ID', 'Geography',
       'Households', '0-25k', '25k-50k', '50k-75k', '75k-100k', '100k-125k',
       '125k-150k', '> 150k', 'index_right', 'NTAName'],
      dtype='object')

In [11]:
# Saving to file as .shp
## Impt to set encoding = 'utf-8' or it will throw a codec error
MN_Yelp_NTA_joined.to_file(path_yelp + 'MN/MN_Yelp_CensusTract_NTA.shp', driver='ESRI Shapefile', encoding = 'utf-8')
BK_Yelp_NTA_joined.to_file(path_yelp + 'BK/BK_Yelp_CensusTract_NTA.shp', driver='ESRI Shapefile', encoding = 'utf-8')
